<a href="https://colab.research.google.com/github/anirudhprabhakaran3/mlforqc/blob/calvin_neural_layout/NeuralLayout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd

#Lets go, Keras!

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Activation, InputLayer
from keras.layers.core import Dense, Flatten

In [ ]:
x_train = pd.read_csv("/content/drive/MyDrive/MLforQC/Athens_x_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/MLforQC/Athens_y_train.csv")

x_test = pd.read_csv("/content/drive/MyDrive/MLforQC/Athens_x_test.csv")
y_test = pd.read_csv("/content/drive/MyDrive/MLforQC/Athens_y_test.csv")

In [ ]:
y_train.head()

,0,1,2,3,4
0,4.0,3.0,2.0,1.0,0.0
1,4.0,1.0,0.0,3.0,2.0
2,1.0,0.0,2.0,3.0,4.0
3,4.0,3.0,2.0,1.0,0.0
4,1.0,2.0,3.0,4.0,0.0


### Defining a one-hot-encoding function

In [ ]:
def oneHotEncodeDF(df):
  for qbit in range(5):
    for oneHotValues in range(5):
      # We make a column for each of the 5 one_hot values, per qubit
      column = []
      for row in range(len(df)):
        if df[str(qbit)][row]==oneHotValues:
          column.append(1)
        else:
          column.append(0)

      # Add the column to the dataframe
      df[str(qbit)+"_"+str(oneHotValues)] = column

  # Droping the initial five columns
  df.drop(['0', '1', '2', '3', '4'], axis=1, inplace=True)


### Defining a Repair Operator

In [ ]:
# Input list has length 5x5 = 25

# This function uses the repair operator approach
def get_final_outputs(arr):
  assert len(arr)==25

  final_array = [-1, -1, -1, -1 ,-1]

  # Splitting the list
  q0 = arr[0:5]
  q1 = arr[5:10]
  q2 = arr[10:15]
  q3 = arr[15:20]
  q4 = arr[20:25]

  # List of lists
  original = [q0, q1, q2, q3, q4]
  list_partitions = [q0, q1, q2, q3, q4]

  # Each iteration in while decides an output qbit for one of the 5 input qbits
  while(len(list_partitions)!=0):
    maxim = -1
    for iter in list_partitions:
      maxim=max(maxim, max(iter))

    for given_list in list_partitions:
      if maxim == max(given_list):
        pos = given_list.index(maxim)
        final_array[original.index(given_list)] = pos

        # removing that list and the possibility of another list getting the same qubit index
        list_partitions.remove(given_list)
        for iter in list_partitions:
          iter[pos] = -1
        break

  return final_array



In [ ]:
# Testing the repair operator
foo = [0.15, 0.23, 0.12 ,0.06, 0.4, 0.08, 0.16, 0.25, 0.13, 0.2, 0.05, 0.06, 0.31, 0.33, 0.2, 0.45, 0.1, 0.03, 0.15, 0.17, 0.27, 0.2, 0.18, 0.06, 0.18]
print(get_final_outputs(foo))
# output: [4, 2, 3, 0, 1]

## Final NN

In [ ]:
# Data elements have 78 features

# Output layer has 25 neurons (5x5)
# for each of the 5 circuit qbits, we have 5 probabilities corresponding to the 5 processor qbits

neuralLayout = Sequential([
    InputLayer(input_shape=(78)),
    Dense(264, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(26, activation='relu')
])

In [ ]:
# One hot encoding the label vector and feeding the data into the neural network
training_labels = y_train.copy()

oneHotEncodeDF(training_labels)

neuralLayout.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
neuralLayout.fit(x_train, training_labels, validation_split=0.2, batch_size=40, epochs=10, verbose=2)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
x_train

In [ ]:
training_labels

,0_0,0_1,0_2,0_3,0_4,1_0,1_1,1_2,1_3,1_4,...,3_0,3_1,3_2,3_3,3_4,4_0,4_1,4_2,4_3,4_4
0,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21401,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
21402,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
21403,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
21404,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0


In [ ]:
y_train.shape

(21406, 5)

In [ ]:
neuralLayout.fit(x_train, y_train, epochs=10)